In [175]:
# https://github.com/llSourcell/pytorch_in_5_minutes/blob/master/demo.py

import torch
from torch.autograd import Variable
import pandas as pd
from random import randint
from ast import literal_eval

# dtype = torch.cuda.FloatTensor # Uncomment this to run on GPU

In [228]:
#read in relevant data
trainData = np.loadtxt('data/trainData.txt', dtype=np.float32)
queryData = np.loadtxt('data/queryData.txt', dtype=np.float32)
df =  pd.read_pickle("./data/KNN.pkl")

In [479]:
K = 5
def generateTripplet(index):
#     point = torch.rand(5)
#     pos = torch.rand(5)
#     neg = torch.rand(5)
    point = torch.tensor(queryData[index])
    pos = torch.tensor(trainData[df.iloc[index].KNN[randint(0,K)]])
    neg = torch.tensor(trainData[df.iloc[index].KNN[randint(K, len(df)-1)]])
    return point, pos, neg

In [480]:
# BATCH_SIZE is batch size; INPUT_D is input dimension; HIDDEN_D is hidden dimension; 
BATCH_SIZE, INPUT_D, HIDDEN_D = 1, 192, 128
ALPHA = 0.5
LEARNING_RATE = 1e-4

# Create random Tensors to hold input and outputs, and wrap them in Variables.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Variables during the backward pass.
x = Variable(torch.randn(INPUT_D).type(dtype), requires_grad=False)

# Create random Tensor for weights, and wrap them in Variables.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Variables during the backward pass.
anchors = Variable(torch.randn(HIDDEN_D, INPUT_D).type(dtype), requires_grad=True)
biases = Variable(torch.randn(INPUT_D).type(dtype), requires_grad=True)

In [481]:
def forward_pass(query):
    return torch.sign(torch.norm(query - anchors, 2) + biases)

In [485]:
for index in range(1):
    query, pos, neg = generateTripplet(index)
    
    queryMapped, posMapped, negMapped = [forward_pass(x) for x in [query, pos, neg]]
    
    # Compute and print loss using operations on Variables.
    # Now loss is a Variable of shape (1,) and loss.data is a Tensor of shape
    # (1,); loss.data[0] is a scalar value holding the loss.
    pos_neg_diff = (torch.norm(queryMapped-posMapped, 2, 0).pow(2) - torch.norm(queryMapped-negMapped, 2, 0).pow(2) + ALPHA)
    loss = nn.functional.relu(pos_neg_diff).sum()
    print(index, loss.data)

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Variables with requires_grad=True.
    # After this we can call var.grad on variables
    loss.backward()

    # Update weights using gradient descent
    biases.data -= LEARNING_RATE * biases.grad.data
    anchors.data -= LEARNING_RATE * anchors.grad.data

    # Manually zero the gradients 
    biases.grad.data.zero_()
    anchors.grad.data.zero_()

0 tensor(0.5000)


In [486]:
t0 = torch.tensor([1.,2.,3.])
t1 = torch.tensor([0.,1.,2.])
t2 = torch.tensor([[2.,3.,4.], [5.,6.,7.]], requires_grad = True)
t3 = torch.sign(torch.norm(t1 - t2, 2, 1) + biases[0:2])